In [0]:
service_credential = dbutils.secrets.get(scope="ecomm-kv-scope",key="ecomm-secret")

spark.conf.set("fs.azure.account.auth.type.ecommadls.dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type.ecommadls.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id.ecommadls.dfs.core.windows.net", "3b059ba0-2363-42d4-345890-37UC3dd866ea7")
spark.conf.set("fs.azure.account.oauth2.client.secret.ecommadls.dfs.core.windows.net", service_credential)
spark.conf.set("fs.azure.account.oauth2.client.endpoint.ecommadls.dfs.core.windows.net", "https://login.microsoftonline.com/2535435251-7BG9-4c72-905f-39bf434340a8a84/oauth2/token")

In [0]:
root = "abfss://zone-2@ecommadls.dfs.core.windows.net"
silver_users     = f"{root}/silver/users"
silver_buyers    = f"{root}/silver/buyers"
silver_sellers   = f"{root}/silver/sellers"
silver_countries = f"{root}/silver/countries"
gold_output      = f"{root}/gold/ecom_one_big_table"

In [0]:
u = spark.read.format("delta").load(silver_users)
b = spark.read.format("delta").load(silver_buyers)
s = spark.read.format("delta").load(silver_sellers)
c = spark.read.format("delta").load(silver_countries)

In [0]:
big = (u.join(c, ["country"], "outer")
        .join(b, ["country"], "outer")
        .join(s, ["country"], "outer"))

In [0]:
# ---------- Select & Alias ----------
goldDF = big.select(
    # from users
    u["country"].alias("Country"),
    u["productsSold"].alias("Users_productsSold"),
    u["productswished"].alias("Users_productsWishhed"),
    u["account_age_years"].alias("Users_account_age_years"),
    u["account_age_group"].alias("Users_account_age_group"),
    u["hasanyapp"].alias("Users_hasanyapp"),
    u["socialnbfollowers"].alias("Users_socialnbfollowers"),
    u["flag_long_title"].alias("Users_flag_long_title"),

    # from countries
    c["sellers"].alias("Countries_Sellers"),
    c["topsellers"].alias("Countries_TopSellers"),
    c["femalesellers"].alias("Countries_FemaleSellers"),
    c["malesellers"].alias("Countries_MaleSellers"),
    c["topfemalesellers"].alias("Countries_TopFemaleSellers"),
    c["topmalesellers"].alias("Countries_TopMaleSellers"),

    # from buyers
    b["buyers"].alias("Buyers_Total"),
    b["topbuyers"].alias("Buyers_Top"),
    b["femalebuyers"].alias("Buyers_Female"),
    b["malebuyers"].alias("Buyers_Male"),
    b["topfemalebuyers"].alias("Buyers_TopFemale"),
    b["topmalebuyers"].alias("Buyers_TopMale"),

    # from sellers
    s["nbsellers"].alias("Sellers_Total"),
    s["sex"].alias("Sellers_Sex"),
    s["meanproductssold"].alias("Sellers_MeanProductsSold"),
    s["meanproductslisted"].alias("Sellers_MeanProductsListed")
)

In [0]:
goldDF.show(5)

+-------+------------------+---------------------+-----------------------+-----------------------+---------------+-----------------------+---------------------+-----------------+--------------------+-----------------------+---------------------+--------------------------+------------------------+------------+----------+-------------+-----------+----------------+--------------+-------------+-----------+------------------------+--------------------------+
|Country|Users_productsSold|Users_productsWishhed|Users_account_age_years|Users_account_age_group|Users_hasanyapp|Users_socialnbfollowers|Users_flag_long_title|Countries_Sellers|Countries_TopSellers|Countries_FemaleSellers|Countries_MaleSellers|Countries_TopFemaleSellers|Countries_TopMaleSellers|Buyers_Total|Buyers_Top|Buyers_Female|Buyers_Male|Buyers_TopFemale|Buyers_TopMale|Sellers_Total|Sellers_Sex|Sellers_MeanProductsSold|Sellers_MeanProductsListed|
+-------+------------------+---------------------+-----------------------+----------

In [0]:
goldDF.write.format("delta").mode("overwrite").save(gold_output)